In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
import gzip
import os
import copy
os.environ["CUDA_VISIBLE_DEVICES"]="0" #for training on gpu

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
from fashion_mnist.utils.mnist_reader import load_mnist

In [4]:
total_train_image, total_train_labels = load_mnist('./fashion_mnist/data/fashion/', kind='train')
test_image, test_labels = load_mnist('./fashion_mnist/data/fashion/', kind='t10k')

In [5]:
train_image, validation_image = train_test_split(total_train_image, test_size=0.15, random_state=0)
train_labels, validation_labels = train_test_split(total_train_labels, test_size=0.15, random_state=0)

In [6]:
train_labels_onehot = pd.get_dummies(train_labels)
validation_labels_onehot = pd.get_dummies(validation_labels)
test_labels_onehot = pd.get_dummies(test_labels)

In [119]:
def softmax(z):
    exp_num = tf.exp(z)
    sum_div = tf.reduce_sum(exp_num,1, keepdims=True)
    return exp_num / sum_div

def sigmoid_function(z):
    return 1 / (1 + tf.exp(-z))    

def cross_entropy(y, h):
    return tf.reduce_sum(-tf.log(h)*y,1, keepdims=True)

def delta_cross_entropy(y,h):
    return tf.subtract(h,y)
    
def gradient_update(gradient,h):
    return gradient.assign(tf.tensordot(tf.transpose(X), tf.subtract(h, y),1)/tf.cast(tf.shape(h)[0], dtype=tf.float32))

def theta_update(theta, gradient):
    return theta.assign(theta - lr * gradient)

def relu(z):
    return tf.maximum(z, 0)
    
def relu_derivative(x):
    condition = tf.greater(x, 0.)
    return tf.where(condition,tf.ones_like(x),tf.zeros_like(x))

In [120]:
sess.run(softmax([[1.8658,2.2292,2.8204]]))

array([[0.19857652, 0.2855949 , 0.5158286 ]], dtype=float32)

In [122]:
sess.run(cross_entropy([[1.,0.,0]], [[0.19857652, 0.2855949 , 0.5158286 ]]))

array([[1.6165807]], dtype=float32)

In [8]:
lr = 0.01
hidden = 2**9
batch = train_image.shape[0]
epochs = 100
iterations = int(train_image.shape[0]/batch)

In [37]:
#plt.imshow(test_image[30].reshape(28,28))

In [10]:
X = tf.placeholder(tf.float32, shape =([None,28*28]))
y = tf.placeholder(tf.float32, shape =([None, 10]))

In [11]:
theta1 = tf.get_variable("theta1", shape=(28*28,hidden), dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer())

In [12]:
bias1 = tf.get_variable("bias1", shape=(1,hidden), dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer())

In [13]:
gradient1 = tf.get_variable("gradient1", shape=(28*28,hidden), dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer())

In [14]:
theta2 = tf.get_variable("theta2", shape=(hidden,10), dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer())

In [15]:
bias2 = tf.get_variable("bias2", shape=(1,10), dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer())

In [16]:
gradient2 = tf.get_variable("gradient2", shape=(hidden,10), dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer())

In [17]:
z1 = tf.add(tf.matmul(X, theta1), bias1)
h1 = relu(z1)
z2 = tf.add(tf.matmul(h1, theta2), bias2)
h2 = softmax(z2)

In [123]:
dE_dh2 = delta_cross_entropy(y,h2)

In [ ]:
dh2_dz2 = 

In [127]:
dE_dh2_value = sess.run(dE_dh2,feed_dict={X:train_image_norm[index:index+batch], y:train_labels_onehot[index:index+batch]})

In [18]:
diff = tf.subtract(h2, y)

In [19]:
loss = cross_entropy(y,h2)

In [20]:
tf_X = tf.placeholder(tf.float32)
tf_X_norm = tf.image.per_image_standardization(tf_X)

In [21]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.InteractiveSession(config=config)

train_image_norm = sess.run(tf_X_norm, feed_dict={tf_X: train_image})
validation_image_norm = sess.run(tf_X_norm, feed_dict={tf_X: validation_image})
test_image_norm = sess.run(tf_X_norm, feed_dict={tf_X: test_image})

In [125]:
sess.run(tf.global_variables_initializer())

train_epoch_loss_list = []
validation_epoch_loss_list = []

for i in range(epochs):
    batch_accumulate_loss = 0
    
    print('Epoch {}'.format(i))
    for j in range(iterations):
        index = j*batch

        theta_value = sess.run(theta_update(theta,gradient))
        y_values, h_values, gradient_values, train_loss_value = sess.run([y, h, gradient_update(gradient, h), loss], feed_dict={X:train_image_norm[index:index+batch], y:train_labels_onehot[index:index+batch]})
        batch_accumulate_loss += train_loss_value

    train_epoch_loss = batch_accumulate_loss/iterations
    train_epoch_loss_list.append(train_epoch_loss)
    validation_epoch_loss = sess.run(loss, feed_dict={X:validation_image_norm, y:validation_labels_onehot})
    validation_epoch_loss_list.append(validation_epoch_loss)

    print('Iteration loss:')
    print(train_epoch_loss)
    print('Validation loss:')
    print(validation_epoch_loss)

Epoch 0


NameError: name 'theta' is not defined

In [ ]:
fig = plt.figure(1)
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.grid()
plt.xlabel('Epochs')
plt.ylabel('Average Cross-Entropy Loss ')
plt.plot(range(len(train_epoch_loss_list)),train_epoch_loss_list, '-bx')
plt.plot(range(len(validation_epoch_loss_list)),validation_epoch_loss_list, '-r')

plt.show()

In [ ]:
h_values = sess.run(h, feed_dict={X:test_image_norm})

In [ ]:
result_labels = np.zeros_like(h_values,dtype=int)

In [ ]:
result_labels[np.arange(len(h_values)), h_values.argmax(1)] = 1

In [ ]:
result_labels = np.argmax(result_labels, axis=1)

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
f1_score(test_labels,result_labels, average='micro')

In [ ]:
from pandas_ml import ConfusionMatrix

In [ ]:
ConfusionMatrix(test_labels,result_labels)